In [325]:
import pandas as pd 
import numpy as np 
from scipy.optimize import curve_fit
from sklearn.metrics import r2_score
import plotly.express as px
import sys 

In [326]:
import os 
plants = pd.read_csv("G:\生生不息\实验数据\BaiduSyncdisk\Plant_data_halfhour.csv")
whole_time=[pd.to_datetime(day+' '+hour) for day,hour in zip(plants['Date'],plants['Time'])]
plants['time']=whole_time
plants['pixel diff']=plants['Area per plant'].diff(1)
plants=plants.set_index('time')
plants['time'] = plants.index

plants = plants[(plants.index.hour == 22) & (plants.index > '2023-05-01')]

plants['time'] = [date.date() for date in plants.index.unique()]
plants = plants.set_index('time')

plants_day = pd.DataFrame({})
#plants_day['time'] = plants.index.unique()
#plants_day = plants_day.set_index('time')
for i in range(10):
    new_df = pd.DataFrame({})
    new = []
    times =[]
    #print(i)
    for j in range(len(plants)):
        if plants['No.'][j] == i:
            new.append(plants['Area per plant'][j])
            times.append(plants.index[j])
    new_df['plant '+str(i)] = new
    new_df['plant '+str(i)+' diff'] = new_df['plant '+str(i)].diff(1)
    new_df['time'] = times
    #plants_day['time'] = plants.index.unique()
    #plants_day = plants_day.set_index('time')
    plants_day['No. '+str(i)+' area'] = new_df['plant '+str(i)]
plants_day['time'] = new_df['time']
plants_day = plants_day.set_index('time')
plants_day


,No. 0 area,No. 1 area,No. 2 area,No. 3 area,No. 4 area,No. 5 area,No. 6 area,No. 7 area,No. 8 area,No. 9 area
time,,,,,,,,,,
2023-05-01,142.06,0.38,220.56,199.73,5.92,0.29,NaN,224.59,0.04,180.75
2023-05-02,143.18,0.30,221.08,199.76,3.81,0.29,NaN,213.89,0.09,179.90
2023-05-03,143.10,0.39,220.57,199.10,6.37,0.24,NaN,211.38,0.09,179.60
2023-05-09,13.69,13.88,10.92,8.84,17.36,13.48,NaN,11.91,10.75,16.79
2023-05-09,13.97,14.03,11.02,8.92,17.55,13.70,NaN,12.14,10.84,17.20
2023-05-10,19.90,19.56,16.16,12.17,24.55,20.05,NaN,17.31,15.33,23.53
2023-05-11,20.22,19.79,16.37,12.12,24.78,20.33,NaN,17.50,22.80,32.03
2023-05-11,28.64,27.87,24.37,17.41,33.68,28.06,NaN,23.10,23.17,32.30
2023-05-12,29.03,28.02,24.53,17.40,34.11,28.14,NaN,23.36,31.06,39.72


In [416]:

def average_duplicate_rows(df):
    # Group by index and calculate the average for each group
    averaged_df = df.groupby(df.index).mean()
    
    # Reset the index to make it unique
    averaged_df.reset_index(inplace=True)
    
    return averaged_df

plants_day = average_duplicate_rows(plants_day)
plants_day['time'] = pd.to_datetime(plants_day['time'])
plants_day = plants_day.set_index('time')

Unexpected exception formatting exception. Falling back to standard exception


Traceback (most recent call last):
  File "c:\Users\Administrator\AppData\Local\Programs\Python\Python311\Lib\site-packages\pandas\core\groupby\groupby.py", line 1490, in array_func
    to generate the indices of each group in the sorted data and then passes the
                     ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\Administrator\AppData\Local\Programs\Python\Python311\Lib\site-packages\pandas\core\groupby\ops.py", line 959, in _cython_operation
    sorter = np.lexsort((ids, self.indexer))
       ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\Administrator\AppData\Local\Programs\Python\Python311\Lib\site-packages\pandas\core\groupby\ops.py", line 657, in cython_operation
    # e.g. if we are int64 and need to restore to datetime64/timedelta64
       ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\Administrator\AppData\Local\Programs\Python\Python311\Lib\site-packages\pandas\core\groupby\ops.py", line 

In [421]:
plants_day = plants_day.set_index('time')

In [406]:
# Define a function that generates a list of time ranges for a period
def generate_time_ranges(start_time, end_time, own_freq, other_freq):
    time_ranges = []
    while start_time < end_time:
        time_ranges.append((start_time, start_time + own_freq))
        start_time += own_freq + other_freq
    return time_ranges

# Define the start and end times and frequency for each period
light_start_time = pd.Timestamp('2023-05-4 22:00:00')
light_end_time = pd.Timestamp('2023-05-21 22:00:00')    
light_frequency = pd.Timedelta(hours=24)
bug_freq = pd.Timedelta(hours=0)
dark_start_time = pd.Timestamp('2023-05-4 11:00:00')
dark_end_time = pd.Timestamp('2023-05-15 18:00:00')
dark_frequency = pd.Timedelta(hours=7)

#需要补充光质设定的信息，等查博消息

# Generate the time ranges for each period
light_time_ranges = generate_time_ranges(light_start_time, light_end_time, light_frequency,bug_freq)
dark_time_ranges = generate_time_ranges(dark_start_time, dark_end_time, dark_frequency, light_frequency)

In [424]:
csv_path = "./arranged_data4"
files = os.listdir(csv_path)
num_files = len(files)-1
dataframes4 = {}

start = 44
for i in range(num_files): #有效数据
    dataframes4['df'+str(i)] = pd.read_csv('arranged_data4/sub_df'+str(i)+'.csv')

combine_df = dataframes4['df0']
for i in range(start,num_files):
    combine_df=pd.concat((combine_df,dataframes4['df'+str(i)]))

C:\Users\Administrator\AppData\Local\Temp\ipykernel_39796\3281508359.py:8: DtypeWarning:

Columns (40,41,45,46,47,48,49) have mixed types. Specify dtype option on import or set low_memory=False.



In [425]:
comb = combine_df
comb

,Unnamed: 0.1,Unnamed: 0,空调光期温度设定,空调暗期温度设定,空调温度上限设定,空调温度下限设定,新风机手动开度,新风机手自动模式,空调自动模式,户外温度,...,电磁阀手自动选择,电磁阀手动开阀,电磁阀手动关阀,time,平均温度,平均湿度,灯具负荷,自然负荷,新风负荷,总热渗入量
0,0,0,22.0,18.0,18.0,17.0,100.0,False,False,17.925346,...,False,False,False,2023-04-07 15:37:23.109320,19.750000,71.000000,0,-1.301065,-0.118279,-1.419343
1,1,1,22.0,18.0,18.0,17.0,100.0,False,False,17.170139,...,False,False,False,2023-04-07 15:44:30.472855,20.450001,75.800003,0,-2.338697,-0.212609,-2.551306
2,2,2,22.0,18.0,18.0,17.0,100.0,False,False,17.378471,...,False,False,False,2023-04-07 15:45:31.410334,20.450001,76.899998,0,-2.190146,-0.199104,-2.389251
3,3,3,22.0,18.0,18.0,17.0,100.0,False,False,17.635998,...,False,False,False,2023-04-07 15:46:32.357841,20.500000,77.650002,0,-2.042170,-0.185652,-2.227821
4,4,4,22.0,18.0,18.0,17.0,100.0,False,False,17.774883,...,False,False,False,2023-04-07 15:47:33.291620,20.599999,76.950001,0,-2.014442,-0.183131,-2.197573
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
195,46016,46016,24.5,21.6,19.5,18.5,100.0,True,True,13.773148,...,False,False,False,2023-05-13 01:04:19.311677,24.750000,80.500000,0,-7.827017,-0.711547,-8.538564
196,46017,46017,24.5,21.6,19.5,18.5,100.0,True,True,13.773148,...,False,False,False,2023-05-13 01:05:20.415023,24.750000,80.100002,0,-7.827017,-0.711547,-8.538564
197,46018,46018,24.5,21.6,19.5,18.5,100.0,True,True,13.773148,...,False,False,False,2023-05-13 01:06:21.809154,24.750000,79.799999,0,-7.827017,-0.711547,-8.538564
198,46019,46019,24.5,21.6,19.5,18.5,100.0,True,True,13.773148,...,False,False,False,2023-05-13 01:07:23.024541,24.750000,79.500000,0,-7.827017,-0.711547,-8.538564


In [426]:
enviros = comb
comb['time'] = pd.to_datetime(comb['time'])
day_enviros = {}
day_means = {}

for time_range in light_time_ranges:
    enviros_day = enviros[(enviros['time'] >= time_range[0]) & (enviros['time'] < time_range[1])]
    try:
        del enviros_day['Unnamed: 0']
        del enviros_day['Unnamed: 0.1']
    except:
        continue
    day_enviros[time_range[1].date()] = enviros_day
    day_means[time_range[1].date()] = enviros_day.mean()
day_means = pd.DataFrame(day_means).transpose()

In [430]:
test_hetero = pd.concat((plants_day,day_means),axis=1)

In [431]:
test_hetero

,No. 0 area,No. 1 area,No. 2 area,No. 3 area,No. 4 area,No. 5 area,No. 6 area,No. 7 area,No. 8 area,No. 9 area,...,电磁阀手自动选择,电磁阀手动开阀,电磁阀手动关阀,time,平均温度,平均湿度,灯具负荷,自然负荷,新风负荷,总热渗入量
2023-05-01,142.060,0.380,220.560,199.730,5.920,0.290,NaN,224.590,0.040,180.750,...,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN
2023-05-02,143.180,0.300,221.080,199.760,3.810,0.290,NaN,213.890,0.090,179.900,...,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN
2023-05-03,143.100,0.390,220.570,199.100,6.370,0.240,NaN,211.380,0.090,179.600,...,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN
2023-05-09,13.830,13.955,10.970,8.880,17.455,13.590,NaN,12.025,10.795,16.995,...,0.0,0.0,0.0,2023-05-09 09:56:17.441028608,23.380499,60.639761,0.0,-5.058483,-4.996805,-10.055287
2023-05-10,19.900,19.560,16.160,12.170,24.550,20.050,NaN,17.310,15.330,23.530,...,0.0,0.0,0.0,2023-05-10 09:59:40.092431616,23.75528,63.325055,0.0,-5.54488,-4.577341,-10.122221
2023-05-11,24.430,23.830,20.370,14.765,29.230,24.195,NaN,20.300,22.985,32.165,...,0.0,0.0,0.0,2023-05-11 10:02:57.651791616,24.190854,75.192109,0.0,-4.037858,-1.52694,-5.564798
2023-05-12,34.065,32.400,28.840,19.685,38.915,32.005,NaN,26.640,31.165,39.735,...,0.0,0.0,0.0,2023-05-12 09:48:52.685044992,24.346434,75.716471,0.0,-3.119306,-1.46415,-4.583456
2023-05-13,45.170,42.645,37.865,25.090,50.375,40.655,NaN,34.030,40.305,48.700,...,0.0,0.0,0.0,2023-05-12 23:34:24.384818176,24.552717,82.083424,0.0,-6.911786,-2.290513,-9.202299
2023-05-14,56.405,54.785,47.645,31.955,65.045,50.765,NaN,44.015,50.690,61.250,...,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN
2023-05-15,69.060,70.605,59.605,41.855,82.600,63.415,NaN,56.745,66.370,77.140,...,0.0,0.0,0.0,2023-05-15 15:07:58.086491648,24.930964,65.172126,0.0,5.544489,5.55614,11.100629


In [432]:
test_hetero['date'] = pd.to_datetime(test_hetero.index)
test_hetero.sort_values(by='date',inplace=True)
test_hetero

,No. 0 area,No. 1 area,No. 2 area,No. 3 area,No. 4 area,No. 5 area,No. 6 area,No. 7 area,No. 8 area,No. 9 area,...,电磁阀手动开阀,电磁阀手动关阀,time,平均温度,平均湿度,灯具负荷,自然负荷,新风负荷,总热渗入量,date
2023-05-01,142.060,0.380,220.560,199.730,5.920,0.290,NaN,224.590,0.040,180.750,...,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,2023-05-01
2023-05-02,143.180,0.300,221.080,199.760,3.810,0.290,NaN,213.890,0.090,179.900,...,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,2023-05-02
2023-05-03,143.100,0.390,220.570,199.100,6.370,0.240,NaN,211.380,0.090,179.600,...,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,2023-05-03
2023-05-05,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.006349,0.002381,2023-05-05 09:50:17.229921024,23.375556,75.545714,0.0,-1.01725,-0.983506,-2.000757,2023-05-05
2023-05-06,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.009098,0.01213,2023-05-06 09:41:48.894401536,23.949356,73.08097,0.0,-3.778155,-3.04288,-6.821035,2023-05-06
2023-05-07,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.0,0.0,2023-05-07 10:00:09.240391168,23.61953,67.893514,0.0,-6.251918,-6.21565,-12.467568,2023-05-07
2023-05-08,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.0,0.0,2023-05-08 10:00:08.851656960,23.717742,63.06638,0.0,-5.051343,-4.824035,-9.875378,2023-05-08
2023-05-09,13.830,13.955,10.970,8.880,17.455,13.590,NaN,12.025,10.795,16.995,...,0.0,0.0,2023-05-09 09:56:17.441028608,23.380499,60.639761,0.0,-5.058483,-4.996805,-10.055287,2023-05-09
2023-05-10,19.900,19.560,16.160,12.170,24.550,20.050,NaN,17.310,15.330,23.530,...,0.0,0.0,2023-05-10 09:59:40.092431616,23.75528,63.325055,0.0,-5.54488,-4.577341,-10.122221,2023-05-10
2023-05-11,24.430,23.830,20.370,14.765,29.230,24.195,NaN,20.300,22.985,32.165,...,0.0,0.0,2023-05-11 10:02:57.651791616,24.190854,75.192109,0.0,-4.037858,-1.52694,-5.564798,2023-05-11


## 光质数据

In [331]:
from led_models import light_module

In [378]:
lights_set_df = pd.read_excel('light_settings.xlsx')

In [379]:
lights_set_df = lights_set_df.set_index('No.')

In [380]:
lights_set_df

,2023-05-07,2023-05-08,2023-05-09,2023-05-10,2023-05-11,2023-05-12,2023-05-13,2023-05-14,2023-05-15,2023-05-16,...,2023-05-21,2023-05-22,2023-05-23,2023-05-24,2023-05-25,2023-05-26,2023-05-27,2023-05-28,2023-05-29,2023-05-30
No.,,,,,,,,,,,,,,,,,,,,,
4,14W,10W+10R,10W+10R,10W+10R,10W+10R,10W+10R,10W+10R,10W+10R,10W+10R,14W,...,14W,14W,14W,14W,14W,14W,14W,14W,14W,14W
5,14W,10W+10R,10W+10R,10W+10R,10W+10R,10W+10R,10W+10R,10W+10R,10W+10R,14W,...,14W,14W,14W,14W,14W,14W,14W,14W,14W,14W
7,14W,10W+10R,10W+10R,10W+10R,10W+10R,10W+10R,10W+10R,10W+10R,10W+10R,14W,...,14W,14W,14W,14W,14W,14W,14W,14W,14W,14W
0,14W,10W+10R,10W+10R,10W+10R,10W+10R,10W+10R,10W+10R,10W+10R,10W+10R,14W,...,10W+10R,12W,10W+10R,12W,10W+10R,12W,10W+10R,12W,10W+10R,12W
1,14W,10W+10R,10W+10R,10W+10R,10W+10R,10W+10R,10W+10R,10W+10R,10W+10R,14W,...,24R+10B,14W,24R+10B,14W,24R+10B,14W,24R+10B,14W,24R+10B,14W
3,14W,10W+10R,10W+10R,10W+10R,10W+10R,10W+10R,10W+10R,10W+10R,10W+10R,14W,...,14W,14W,14W,14W,14W,14W,14W,14W,14W,14W
9,14W,10W+10R,10W+10R,10W+10R,10W+10R,10W+10R,10W+10R,10W+10R,10W+10R,14W,...,16W,16W,24R+10B,24R+10B,16W,16W,24R+10B,24R+10B,16W,16W
8,14W,10W+10R,10W+10R,10W+10R,10W+10R,10W+10R,10W+10R,10W+10R,10W+10R,14W,...,14W,14W,14W,14W,14W,14W,14W,14W,14W,14W
2,14W,10W+10R,10W+10R,10W+10R,10W+10R,10W+10R,10W+10R,10W+10R,10W+10R,14W,...,12W,12W,10W+10R,10W+10R,12W,12W,10W+10R,10W+10R,12W,12W


In [383]:
lights_set_df = lights_set_df.transpose()
lights_set_df

No.,4,5,7,0,1,3,9,8,2
2023-05-07 00:00:00,14W,14W,14W,14W,14W,14W,14W,14W,14W
2023-05-08 00:00:00,10W+10R,10W+10R,10W+10R,10W+10R,10W+10R,10W+10R,10W+10R,10W+10R,10W+10R
2023-05-09 00:00:00,10W+10R,10W+10R,10W+10R,10W+10R,10W+10R,10W+10R,10W+10R,10W+10R,10W+10R
2023-05-10 00:00:00,10W+10R,10W+10R,10W+10R,10W+10R,10W+10R,10W+10R,10W+10R,10W+10R,10W+10R
2023-05-11 00:00:00,10W+10R,10W+10R,10W+10R,10W+10R,10W+10R,10W+10R,10W+10R,10W+10R,10W+10R
2023-05-12 00:00:00,10W+10R,10W+10R,10W+10R,10W+10R,10W+10R,10W+10R,10W+10R,10W+10R,10W+10R
2023-05-13 00:00:00,10W+10R,10W+10R,10W+10R,10W+10R,10W+10R,10W+10R,10W+10R,10W+10R,10W+10R
2023-05-14 00:00:00,10W+10R,10W+10R,10W+10R,10W+10R,10W+10R,10W+10R,10W+10R,10W+10R,10W+10R
2023-05-15 00:00:00,10W+10R,10W+10R,10W+10R,10W+10R,10W+10R,10W+10R,10W+10R,10W+10R,10W+10R
2023-05-16 00:00:00,14W,14W,14W,14W,14W,14W,14W,14W,14W


In [396]:
import pandas as pd
import re

def extract_number(data):
    number_array = []
    
    for item in data:
        numbers = re.findall(r'\d+', item)
        number_array.append(int(numbers[0]) if len(numbers) > 0 else 0)
        
    return number_array

def light_set_class(df):
    detail_df = {}
    
    for key in df.keys():
        key_str = str(key)
        detail_df[key_str + ' W'] = []
        detail_df[key_str + ' R'] = []
        detail_df[key_str + ' B'] = []
        
        for item in df[key]:
            numbers = extract_number([item])
            
            if len(numbers) > 0:
                detail_df[key_str + ' W'].append(numbers[0])
            else:
                detail_df[key_str + ' W'].append(0)
                
            if len(numbers) > 1:
                detail_df[key_str + ' R'].append(numbers[1])
            else:
                detail_df[key_str + ' R'].append(0)
                
            if len(numbers) > 2:
                detail_df[key_str + ' B'].append(numbers[2])
            else:
                detail_df[key_str + ' B'].append(0)
    
    return pd.DataFrame(detail_df)

'''
# Create an example DataFrame
df = pd.DataFrame({'Data Item': ['10W+10R', '5W+15R', '8W', '20R']})

# Call the function to extract numbers and create a new DataFrame
detail_df = light_set_class(df)

# Print the new DataFrame
print(detail_df)
'''

detail_light_sets = light_set_class(lights_set_df)
detail_light_sets['time'] = lights_set_df.index
detail_light_sets = detail_light_sets.set_index('time')

In [436]:
test_hetero.index = pd.to_datetime(test_hetero.index)
detail_light_sets.index, test_hetero.index

(DatetimeIndex(['2023-05-07', '2023-05-08', '2023-05-09', '2023-05-10',
                '2023-05-11', '2023-05-12', '2023-05-13', '2023-05-14',
                '2023-05-15', '2023-05-16', '2023-05-17', '2023-05-18',
                '2023-05-19', '2023-05-20', '2023-05-21', '2023-05-22',
                '2023-05-23', '2023-05-24', '2023-05-25', '2023-05-26',
                '2023-05-27', '2023-05-28', '2023-05-29', '2023-05-30'],
               dtype='datetime64[ns]', name='time', freq=None),
 DatetimeIndex(['2023-05-01', '2023-05-02', '2023-05-03', '2023-05-05',
                '2023-05-06', '2023-05-07', '2023-05-08', '2023-05-09',
                '2023-05-10', '2023-05-11', '2023-05-12', '2023-05-13',
                '2023-05-14', '2023-05-15', '2023-05-16', '2023-05-17',
                '2023-05-18', '2023-05-19', '2023-05-20', '2023-05-21',
                '2023-05-22'],
               dtype='datetime64[ns]', freq=None))

In [437]:
pd.concat((detail_light_sets,test_hetero),axis=1)

,4 W,4 R,4 B,5 W,5 R,5 B,7 W,7 R,7 B,0 W,...,电磁阀手动开阀,电磁阀手动关阀,time,平均温度,平均湿度,灯具负荷,自然负荷,新风负荷,总热渗入量,date
2023-05-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,2023-05-01
2023-05-02,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,2023-05-02
2023-05-03,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,2023-05-03
2023-05-05,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.006349,0.002381,2023-05-05 09:50:17.229921024,23.375556,75.545714,0.0,-1.01725,-0.983506,-2.000757,2023-05-05
2023-05-06,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.009098,0.01213,2023-05-06 09:41:48.894401536,23.949356,73.08097,0.0,-3.778155,-3.04288,-6.821035,2023-05-06
2023-05-07,14.0,0.0,0.0,14.0,0.0,0.0,14.0,0.0,0.0,14.0,...,0.0,0.0,2023-05-07 10:00:09.240391168,23.61953,67.893514,0.0,-6.251918,-6.21565,-12.467568,2023-05-07
2023-05-08,10.0,0.0,0.0,10.0,0.0,0.0,10.0,0.0,0.0,10.0,...,0.0,0.0,2023-05-08 10:00:08.851656960,23.717742,63.06638,0.0,-5.051343,-4.824035,-9.875378,2023-05-08
2023-05-09,10.0,0.0,0.0,10.0,0.0,0.0,10.0,0.0,0.0,10.0,...,0.0,0.0,2023-05-09 09:56:17.441028608,23.380499,60.639761,0.0,-5.058483,-4.996805,-10.055287,2023-05-09
2023-05-10,10.0,0.0,0.0,10.0,0.0,0.0,10.0,0.0,0.0,10.0,...,0.0,0.0,2023-05-10 09:59:40.092431616,23.75528,63.325055,0.0,-5.54488,-4.577341,-10.122221,2023-05-10
2023-05-11,10.0,0.0,0.0,10.0,0.0,0.0,10.0,0.0,0.0,10.0,...,0.0,0.0,2023-05-11 10:02:57.651791616,24.190854,75.192109,0.0,-4.037858,-1.52694,-5.564798,2023-05-11


In [393]:
test_hetero

,No. 1 area,No. 2 area,No. 3 area,No. 4 area,No. 5 area,No. 6 area,No. 7 area,No. 8 area,No. 9 area,空调光期温度设定,...,电磁阀手动开阀,电磁阀手动关阀,time,平均温度,平均湿度,灯具负荷,自然负荷,新风负荷,总热渗入量,date
2023-05-01,0.38,220.56,199.73,5.92,0.29,NaN,224.59,0.04,180.75,NaN,...,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,2023-05-01
2023-05-02,0.30,221.08,199.76,3.81,0.29,NaN,213.89,0.09,179.90,NaN,...,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,2023-05-02
2023-05-03,0.39,220.57,199.10,6.37,0.24,NaN,211.38,0.09,179.60,NaN,...,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,2023-05-03
2023-05-05,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,24.5,...,0.006349,0.002381,2023-05-05 09:50:17.229921024,23.375556,75.545714,0.0,-1.01725,-0.983506,-2.000757,2023-05-05
2023-05-06,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,24.5,...,0.009098,0.01213,2023-05-06 09:41:48.894401536,23.949356,73.08097,0.0,-3.778155,-3.04288,-6.821035,2023-05-06
2023-05-07,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,24.5,...,0.0,0.0,2023-05-07 10:00:09.240391168,23.61953,67.893514,0.0,-6.251918,-6.21565,-12.467568,2023-05-07
2023-05-08,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,24.5,...,0.0,0.0,2023-05-08 10:00:08.851656960,23.717742,63.06638,0.0,-5.051343,-4.824035,-9.875378,2023-05-08
2023-05-09,14.03,11.02,8.92,17.55,13.70,NaN,12.14,10.84,17.20,NaN,...,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,2023-05-09
2023-05-09,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,24.5,...,0.0,0.0,2023-05-09 09:56:17.441028608,23.380499,60.639761,0.0,-5.058483,-4.996805,-10.055287,2023-05-09
2023-05-09,13.88,10.92,8.84,17.36,13.48,NaN,11.91,10.75,16.79,NaN,...,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,2023-05-09


## 之前尝试

In [27]:
a,b = plants[['Pixel','Area']].mean()

In [28]:
plants['Pixel'].mean()

578192.1160747664

In [39]:
enviros['time'] = pd.to_datetime(enviros['time'])
plants['time'] = pd.to_datetime(plants['time'])

In [120]:
enviros['time'] = pd.to_datetime(enviros['time'])
for time_range in light_time_ranges:
    #time_range = pd.to_datetime(time_range)
    for day in pd.unique(enviros['time'].dt.date):
        #df_day = df[df['time'].dt.date == day]
        enviros_day = enviros[enviros['time'].dt.date == day]
        #df_light = df_day[(df_day['time'] >= time_range[0]) & (df_day['time'] < time_range[1])]
        enviro_light = enviros_day[(enviros_day['time'] >= time_range[0]) & (enviros_day['time'] < time_range[1])]


In [44]:
pixel_diffs, Ts, RHs, blues, reds, greens, CO2_s, ECs, pHs = 9 * [ ]
pixel_diffs

ValueError: not enough values to unpack (expected 9, got 0)

In [109]:
time_range = light_time_ranges[2]
print(time_range)
day = pd.unique(df['time'].dt.date)[0]
print(day)
enviros[enviros['time'].dt.date == day]

(Timestamp('2023-05-06 18:00:00'), Timestamp('2023-05-07 11:00:00'))
2023-04-11


,Unnamed: 0,空调光期温度设定,空调暗期温度设定,空调温度上限设定,空调温度下限设定,新风机手动开度,新风机手自动模式,空调自动模式,户外温度,户外湿度,...,空调开机指令下发,空调手动模式选择,空调风速选择,空调手动温度设定,空调手动操作指令下发,空调手自动切换,电磁阀手自动选择,电磁阀手动开阀,电磁阀手动关阀,time
3646,3646,22.000000,18.0,19.5,18.5,0.0,False,True,17.427662,100.0,...,False,3.0,2.0,24.000000,False,False,False,False,False,2023-04-11 00:00:10.296437
3647,3647,22.000000,18.0,19.5,18.5,0.0,False,True,17.526043,100.0,...,False,3.0,2.0,24.000000,False,False,False,False,False,2023-04-11 00:01:11.284056
3648,3648,22.000000,18.0,19.5,18.5,0.0,False,True,17.427662,100.0,...,False,3.0,2.0,24.000000,False,False,False,False,False,2023-04-11 00:02:12.148145
3649,3649,22.000000,18.0,19.5,18.5,0.0,False,True,17.427662,100.0,...,False,3.0,2.0,24.000000,False,False,False,False,False,2023-04-11 00:03:13.120894
3650,3650,22.000000,18.0,19.5,18.5,0.0,False,True,17.526043,100.0,...,False,3.0,2.0,24.000000,False,False,False,False,False,2023-04-11 00:04:14.026903
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5037,5037,23.700001,18.0,19.5,18.5,100.0,False,True,14.117477,100.0,...,False,1.0,4.0,23.700001,False,False,False,False,False,2023-04-11 23:55:43.903372
5038,5038,23.600000,18.0,19.5,18.5,100.0,False,True,14.166668,100.0,...,False,1.0,6.0,23.600000,False,False,False,False,False,2023-04-11 23:56:44.991006
5039,5039,23.600000,18.0,19.5,18.5,100.0,False,True,14.068287,100.0,...,False,1.0,6.0,23.600000,False,False,False,False,False,2023-04-11 23:57:45.991275
5040,5040,23.600000,18.0,19.5,18.5,100.0,False,True,14.068287,100.0,...,False,1.0,6.0,23.600000,False,False,False,False,False,2023-04-11 23:58:47.056068


In [110]:
#大整体，假设大家光质没区别   

df = plants

def time_range_gather(df,enviros, time_ranges):
    days = []
    pixel_diffs = []
    Ts = []
    RHs = []
    blues = []
    reds = []
    greens = []
    CO2_s = []
    ECs = []
    pHs = []
    # Iterate over the periods and time ranges and extract the data for each time range separately
    for time_range in time_ranges:
        print(time_range)
        #time_range = pd.to_datetime(time_range)
        for day in pd.unique(df['time'].dt.date):
            #print(time_range)
            start_date = pd.to_datetime(time_range[0]).date()
            end_date = pd.to_datetime(time_range[1]).date()
            if day >= start_date and day <= end_date:
                print(day)
                df_day = df[df['time'].dt.date == day]
                enviros_day = enviros[enviros['time'].dt.date == day]
                df_light = df_day[(df_day['time'] >= time_range[0]) & (df_day['time'] < time_range[1])]
                enviro_light = enviros_day[(enviros_day['time'] >= time_range[0]) & (enviros_day['time'] < time_range[1])]
                days.append(day)
                pixel_diffs.append(df_light['pixel diff'].mean())

                print(enviros_day)
                print(enviro_light)
                
                T, RH, blue, red, green, co2, ec, ph = enviro_light[['2号室内温度','2号室内湿度','1号蓝比','1号红比','1号绿比','1号室内CO2浓度','营养液EC','营养液PH']].mean()
                Ts.append(T)
                RHs.append(RH)
                blues.append(blue)
                reds.append(red)
                greens.append(green)
                CO2_s.append(co2)
                ECs.append(ec)
                pHs.append(ph)        

    return days,pixel_diffs, Ts, RHs, blues, reds, greens, CO2_s, ECs, pHs

days_light,pixel_diffs_light,Ts_light, RHs_light, blues_light, reds_light, greens_light, CO2_s_light, ECs_light, pHs_light = time_range_gather(df,enviros,light_time_ranges)

(Timestamp('2023-05-04 18:00:00'), Timestamp('2023-05-05 11:00:00'))
2023-05-04
2023-05-04 18:00:00
Empty DataFrame
Columns: [Unnamed: 0, 空调光期温度设定, 空调暗期温度设定, 空调温度上限设定, 空调温度下限设定, 新风机手动开度, 新风机手自动模式, 空调自动模式, 户外温度, 户外湿度, 1号室内温度, 1号室内湿度, 1号室内CO2浓度, 2号室内温度, 2号室内湿度, 2号室内CO2浓度, 1号PPFD, 1号色温, 1号PAR, 1号红比, 1号绿比, 1号蓝比, 2号PPFD, 2号色温, 2号PAR, 2号红比, 2号绿比, 2号蓝比, 3号PPFD, 3号色温, 3号PAR, 3号红比, 3号绿比, 3号蓝比, 1号风速, 2号风速, 营养液EC, 营养液PH, 营养液液温, 空调手动开关机, 空调开机指令下发, 空调手动模式选择, 空调风速选择, 空调手动温度设定, 空调手动操作指令下发, 空调手自动切换, 电磁阀手自动选择, 电磁阀手动开阀, 电磁阀手动关阀, time]
Index: []

[0 rows x 50 columns]
Empty DataFrame
Columns: [Unnamed: 0, 空调光期温度设定, 空调暗期温度设定, 空调温度上限设定, 空调温度下限设定, 新风机手动开度, 新风机手自动模式, 空调自动模式, 户外温度, 户外湿度, 1号室内温度, 1号室内湿度, 1号室内CO2浓度, 2号室内温度, 2号室内湿度, 2号室内CO2浓度, 1号PPFD, 1号色温, 1号PAR, 1号红比, 1号绿比, 1号蓝比, 2号PPFD, 2号色温, 2号PAR, 2号红比, 2号绿比, 2号蓝比, 3号PPFD, 3号色温, 3号PAR, 3号红比, 3号绿比, 3号蓝比, 1号风速, 2号风速, 营养液EC, 营养液PH, 营养液液温, 空调手动开关机, 空调开机指令下发, 空调手动模式选择, 空调风速选择, 空调手动温度设定, 空调手动操作指令下发, 空调手自动切换, 电磁阀手自动选择, 电磁阀手动开阀, 电磁阀手动关阀, time]
Index: []

[0 rows x 

In [96]:
enviros_day['time'],time_range

(10719   2023-05-15 08:14:03.399347
 10720   2023-05-15 08:15:04.711344
 10721   2023-05-15 08:16:06.102268
 10722   2023-05-15 08:17:07.423242
 10723   2023-05-15 08:18:09.125196
                    ...            
 10904   2023-05-15 11:25:16.684238
 10905   2023-05-15 11:26:17.723965
 10906   2023-05-15 11:27:18.643012
 10907   2023-05-15 11:28:19.592977
 10908   2023-05-15 11:29:20.644671
 Name: time, Length: 190, dtype: datetime64[ns],
 (Timestamp('2023-05-14 18:00:00'), Timestamp('2023-05-15 11:00:00')))

In [92]:
pixel_diffs_light,Ts_light

([-0.344444444444447,
  -18.64222222222222,
  0.05534653465346533,
  0.0015972222222222251,
  0.06777777777777778,
  0.0030718954248365937,
  0.0714814814814815,
  -0.004577464788732409,
  0.019578947368421085,
  0.040144927536231875,
  0.09166666666666666,
  0.005833333333333357,
  0.006509433962264331,
  0.10034246575342465],
 [nan,
  nan,
  nan,
  nan,
  nan,
  nan,
  nan,
  nan,
  nan,
  nan,
  nan,
  nan,
  nan,
  24.286335370555427])

In [56]:
df['pixel diff']

time
2023-04-11 20:38:16      NaN
2023-04-11 20:38:26   -19.59
2023-04-11 20:38:38    39.64
2023-04-11 20:38:50   -29.69
2023-04-11 20:39:01    15.67
                       ...  
2023-05-15 08:45:55    39.65
2023-05-15 08:46:16   -18.51
2023-05-15 08:46:36    -7.07
2023-05-15 08:46:57     3.90
2023-05-15 08:47:18     6.10
Name: pixel diff, Length: 5350, dtype: float64

In [3]:
plants
for i in range(len(plants))

,Date,Time,Group,Pixel,Area,Plant,Area per plant,No.,pixel diff,time
time,,,,,,,,,,
2023-04-11 20:38:16,2023/4/11,20:38:16,2-lower,914767,1533.19,12,127.77,0,NaN,2023-04-11 20:38:16
2023-04-11 20:38:26,2023/4/11,20:38:26,2-mid,719009,1190.03,11,108.18,1,-19.59,2023-04-11 20:38:26
2023-04-11 20:38:38,2023/4/11,20:38:38,1-upper,827799,1478.21,10,147.82,2,39.64,2023-04-11 20:38:38
2023-04-11 20:38:50,2023/4/11,20:38:50,2-upper,727688,1299.44,11,118.13,3,-29.69,2023-04-11 20:38:50
2023-04-11 20:39:01,2023/4/11,20:39:01,3-lower,788481,1338.05,10,133.80,4,15.67,2023-04-11 20:39:01
...,...,...,...,...,...,...,...,...,...,...
2023-05-15 08:45:55,2023-05-15,08:45:55,3-lower,477366,810.09,10,81.01,4,39.65,2023-05-15 08:45:55
2023-05-15 08:46:16,2023-05-15,08:46:16,3-mid,447563,750.02,12,62.50,5,-18.51,2023-05-15 08:46:16
2023-05-15 08:46:36,2023-05-15,08:46:36,3-upper,258179,498.91,9,55.43,7,-7.07,2023-05-15 08:46:36


In [2]:

df_collection = []

for i in range(1,10):
    new_df = pd.DataFrame({})
    new = []
    times = []
    #print(i)
    for j in range(len(plants)):
        if plants['No.'][j] == i:
            new.append(plants['Area per plant'][j])
            times.append(plants['time'][j])
    new_df['plant '+str(i)] = new
    new_df['plant '+str(i)+' diff'] = new_df['plant '+str(i)].diff(1)
    new_df['time'] = times 
    new_df['time'] = pd.to_datetime(new_df['time'])

    try:
        #new_df.resample('1H').mean()
        df_collection.append(new_df)
    except:
        pass
dataframes4 = {}
csv_path = "arranged_data4"
files = os.listdir(csv_path)
num_files = len(files)
print(num_files)
for i in range(num_files):
    dataframes4['df'+str(i)] = pd.read_csv("arranged_data4/sub_df"+str(i)+".csv")

combine_df = pd.concat([dataframes4[key] for key in dataframes4.keys()],ignore_index=True)

merged_df = combine_df
merged_df['time'] = pd.to_datetime(merged_df['time'])
for i in range(len(df_collection)):
    print(i)
    df_collection[i]['time'] = pd.to_datetime(df_collection[i]['time'])
    #ValueError: Merge keys contain null values on left side
    
    print(len(merged_df['time'])-merged_df['time'].count())
    print(len(merged_df['time'])-df_collection[i]['time'].count())
    merged_df = pd.merge_asof(merged_df,df_collection[i],on='time',direction='nearest')
merged_df = merged_df.set_index('time')
daily_df = merged_df.resample('1D').mean()#1天数据
three_hour_df = merged_df.resample('3H').mean()#3小时数据
half_hour_df = merged_df.resample('30min').mean()#半小时数据

hetero_collection = []

67
0
1
56261


ValueError: Merge keys contain null values on left side

In [35]:
from data_upload import data_pretreatment
df = pd.read_csv("G:\生生不息\实验数据\BaiduSyncdisk\dataframe4.csv")
data_pretreatment(4,df)

plants = pd.read_csv("G:\生生不息\实验数据\BaiduSyncdisk\Plant_data_halfhour.csv")
whole_time=[pd.to_datetime(day+' '+hour) for day,hour in zip(plants['Date'],plants['Time'])]
plants['time']=whole_time
plants['pixel diff']=plants['Area per plant'].diff(1)
plants=plants.set_index('time')
plants['time'] = plants.index
#plants = plants[:-1]
re_arranged_plants = pd.DataFrame({})
df_collection = []

for i in range(1,10):
    new_df = pd.DataFrame({})
    new = []
    times = []
    #print(i)
    for j in range(len(plants)):
        if plants['No.'][j] == i:
            new.append(plants['Area per plant'][j])
            times.append(plants['time'][j])
    new_df['plant '+str(i)] = new
    new_df['plant '+str(i)+' diff'] = new_df['plant '+str(i)].diff(1)
    new_df['time'] = times 
    new_df['time'] = pd.to_datetime(new_df['time'])
    #new_df = new_df.set_index('time')
    #new_df.index = pd.to_datetime(new_df.index)
    #ew_df = new_df.resample('30m').mean()
    try:
        #new_df.resample('1H').mean()
        df_collection.append(new_df)
    except:
        pass
from glob import glob
import os 

dataframes4 = {}
csv_path = "arranged_data4"
files = os.listdir(csv_path)
num_files = len(files)

for i in range(num_files):
    dataframes4['df'+str(i)] = pd.read_csv("./arranged_data4/sub_df"+str(i)+".csv")

combine_df = pd.concat([dataframes4[key] for key in dataframes4.keys()],ignore_index=True)
#for i in range(11,33):
#    combine_df=pd.concat([combine_df,dataframes4['df'+str(i)]],axis=1)

merged_df = combine_df
merged_df['time'] = pd.to_datetime(merged_df['time'])
for i in range(len(df_collection)):
    print(i)
    df_collection[i]['time'] = pd.to_datetime(df_collection[i]['time'])
    merged_df = pd.merge_asof(merged_df,df_collection[i],on='time',direction='nearest')

0
1
2
3
4
5
6
7
8
